In [1]:
import os
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
dirpath = os.getcwd()
features_path = r"C:\Users\marie\rep_codes\udder_project\udder_processing\features_dict\gmfeature_table.csv"
data_path = r"C:\Users\marie\rep_codes\udder_project\delpro_vms\animal_info.csv"
miss_path = r"C:\Users\marie\rep_codes\udder_project\delpro_vms\missing_teats.csv"
out_path = os.path.join(dirpath, "teat_len_dist")

data1 = pd.read_csv(features_path)
data2 = pd.read_csv(data_path, sep = ";")[["Animal Number", "Lactation Number"]]

data2.columns = ["cow", "parity"]
data = data1.merge(data2, on = "cow", how = "left")
cols = [col for col in data.columns if col.endswith("len")]
data = data[["cow", "parity"] + cols]

In [3]:
mt_df = pd.read_csv(miss_path)
mt_df["teat"] = [teat + "_len" for teat in mt_df["teat"]]
mt_df["y"] = 1
mt_df = mt_df.rename(columns={"teat": "variable"})

In [4]:
melted_df = pd.melt(data, id_vars=['cow', 'parity'], value_vars=cols)
melted_df2 = pd.merge(melted_df, mt_df, on = ["cow", "variable"], how = "left")
melted_df2["rem"] = np.isnan(melted_df2.y) & np.isnan(melted_df2.value)

In [7]:
gdf = melted_df2.groupby(["parity", "variable"]).agg({'value' : ['mean', 'std',  'min', 'max', 'median', 'count'], 'y' : ['sum'], 'rem': ['sum'], 'cow': ['count']}).reset_index()
gdf.columns = ["_".join(col) if len(col[1]) > 0 else col[0] for col in gdf.columns]
gdf = gdf.rename(columns = {"y_sum": "teat_missing", "rem_sum":"extreme_value_removed"})
gdf.to_csv(os.path.join(out_path, "teat_length_summary_parity.csv"), index = False)

In [8]:
gdf = melted_df2.groupby(["variable"]).agg({'value' : ['mean', 'std',  'min', 'max', 'median', 'count'], 'y' : ['sum'], 'rem': ['sum'], 'cow': ['count']}).reset_index()
gdf.columns = ["_".join(col) if len(col[1]) > 0 else col[0] for col in gdf.columns]
gdf = gdf.rename(columns = {"y_sum": "teat_missing", "rem_sum":"extreme_value_removed"})
gdf.to_csv(os.path.join(out_path, "teat_length_summary.csv"), index = False)